# Chapter 19: Training and Deploying TensorFlow Models at Scale
*Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow*

## 1. Exporting SavedModel

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os

# Model sederhana untuk contoh
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(x_train, y_train, epochs=5)  # Load your training data

# Save model
model_name = "my_mnist_model"
version = "0001"
model_path = os.path.join(model_name, version)
tf.saved_model.save(model, model_path)

## 2. Serving the Model via REST API using TensorFlow Serving

In [ ]:
# Jalankan ini di terminal, bukan di notebook:
# docker pull tensorflow/serving
# docker run -it --rm -p 8501:8501 \
#    -v "$PWD/my_mnist_model:/models/my_mnist_model" \
#    -e MODEL_NAME=my_mnist_model tensorflow/serving

### Mengirim permintaan ke REST API

In [ ]:
import numpy as np
import json
import requests

# X_new = load_your_test_data()  # Shape: (n_samples, 28, 28)
# input_data_json = json.dumps({"signature_name": "serving_default", "instances": X_new.tolist()})
# response = requests.post("http://localhost:8501/v1/models/my_mnist_model:predict", data=input_data_json)
# y_proba = np.array(response.json()["predictions"])

## 3. Convert to TensorFlow Lite for Mobile Deployment

In [ ]:
# Convert to TFLite format
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]  # Optional: quantization
tflite_model = converter.convert()

with open("converted_model.tflite", "wb") as f:
    f.write(tflite_model)

## 4. Model untuk Web menggunakan TensorFlow.js (opsional)
Gunakan perintah terminal:
```
tensorflowjs_converter --input_format=tf_saved_model my_mnist_model/0001 web_model/
```

## 5. Cek GPU dan Set Memori GPU

In [ ]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Atur limit GPU memory (misalnya 2048 MB)
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_virtual_device_configuration(
        gpu,
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

## 6. Distributed Training Strategy

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # model.fit(x_train, y_train, epochs=5)